In [1]:
import os
import ipywidgets as widgets
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyro
import pyro.distributions as dist
# gui
#import qgrid
# pyro & pytorch
import torch
from ipywidgets import fixed, interact, interactive
from ipyfilechooser import FileChooser
from tqdm import tqdm
from scipy.io import loadmat
from sklearn.metrics import matthews_corrcoef, confusion_matrix

from cosmos.utils.visualize import view_parameters, view_glimpse, view_m_probs, view_aoi, view_single_aoi, view_globals
from cosmos.utils.aoi_reader import ReadAoi

%matplotlib inline
#qgrid.enable()

%load_ext tensorboard
%load_ext autoreload
%autoreload 2

In [2]:
device = torch.device("cpu")

def tb_view(path):
    path = os.path.join(path, "scalar")
    %tensorboard --logdir $path

In [3]:
def load_dataset(chooser):
    global data, control
    if chooser.selected is not None:
        data, control = ReadAoi(device=device, path=chooser.selected_path)

        results = FileChooser(chooser.selected_path)
        with fc:
            clear_output()
            display(results)

        # Show or hide folder icons
        results.use_dir_icons = True

        # Change the title (use '' to hide)
        results.title = "<b>Choose Results Folder</b>"

        results.register_callback(run_app)

def run_app(chooser):
    if chooser.selected is not None:
        pyro.clear_param_store()
        pyro.get_param_store().load(
            filename=os.path.join(chooser.selected_path, "params"),
            map_location=device)

        data.predictions = pd.read_csv(
            os.path.join(chooser.selected_path, "predictions.csv"),
            index_col=["aoi", "frame"])

    aoi_widget = widgets.Dropdown(
        options=data.target.index
    )
    frame_widget = widgets.Dropdown(
        options=data.drift.index, value=data.drift.index[0]
    )
    frames_widget = widgets.IntRangeSlider(
        value=[data.drift.index[0], data.drift.index[-1]], min=data.drift.index[0],
        max=data.drift.index[-1], continuous_update=False,
        layout=widgets.Layout(width="100%")
    )
        
    params_widget = widgets.SelectMultiple(
        options=["intensity", "x", "y", "width", "background"],
        value=["intensity", "x", "y", "width"],
        #rows=10,
        description="Parameters",
        disabled=False
    )
    
    
    acc = {}
    acc["All"] = data.labels
    acc["FN"] = data.labels[(data.labels["label"] == 1) & (data.predictions["binary"] == 0)]
    acc["FP"] = data.labels[(data.labels["label"] == 0) & (data.predictions["binary"] == 1)]
    acc["TN"] = data.labels[(data.labels["label"] == 0) & (data.predictions["binary"] == 0)]
    acc["TP"] = data.labels[(data.labels["label"] == 1) & (data.predictions["binary"] == 1)]
    
    acc_widget = widgets.Dropdown(
        options=["FN", "FP", "TN", "TP"],
        value="FN",
    )
    acc_aoi_widget = widgets.Dropdown(
        options=acc[acc_widget.value].index.get_level_values("aoi").unique()
    )
    acc_frame_widget = widgets.Dropdown(
        options=acc[acc_widget.value].loc[int(acc_aoi_widget.value)].index
    )

    def update_aoi_widget(*args): # *args represent zero (case here) or more arguments.
        acc_aoi_widget.options = acc[acc_widget.value].index.get_level_values("aoi").unique()
        acc_aoi_widget.value = acc[acc_widget.value].index.get_level_values("aoi").unique()[0]
        
    def update_frame_widget(*args): # *args represent zero (case here) or more arguments.
        acc_frame_widget.options = acc[acc_widget.value].loc[int(acc_aoi_widget.value)].index
        acc_frame_widget.value = acc[acc_widget.value].loc[int(acc_aoi_widget.value)].index[0]
    
    acc_aoi_widget.observe(update_frame_widget)
    acc_widget.observe(update_aoi_widget)

    w1 = interactive(view_m_probs, aoi=aoi_widget, data=fixed(data), frames=frames_widget,
            m=False, z=False, sp=False, labels=True, prefix="d")


    w2 = interactive(view_parameters, aoi=aoi_widget, data=fixed(data), frames=frames_widget,
            m=False, params=params_widget, prefix="d")

    w3 = interactive(view_aoi, aoi=aoi_widget, frame=frame_widget, data=fixed(data),
             z=True, sp=False, labels=True, target=False, prefix="d")
    
    w4 = interactive(view_single_aoi, aoi=acc_aoi_widget, frame=acc_frame_widget, data=fixed(data),
             z=True, sp=False, labels=True, target=False, acc=acc_widget, prefix="d")
    
    w5 = interactive(view_globals, z=True, j=True, data=fixed(data))
    
    w6 = interactive(tb_view, path=fixed(chooser.selected_path))
    
    tab = widgets.Tab()
    tab.children = [w1, w2, w3, w4, w5, w6]
    tab.titles = ["m_probs", "params", "images", "image", "globals", "tensorboard"]
    for i, title in enumerate(tab.titles):
        tab.set_title(i, title)
    with app:
        clear_output()
        display(tab)

In [4]:
# Create and display a FileChooser widget
dataset = FileChooser(".")
display(dataset)

# Show or hide folder icons
dataset.use_dir_icons = True

# Change the title (use '' to hide)
dataset.title = "<b>Choose Dataset Folder</b>"

dataset.register_callback(load_dataset)

fc = widgets.Output()
app = widgets.Output()
display(fc)
display(app)

FileChooser(path='.', filename='', show_hidden='False')

Output()

Output()

In [2]:
# from tensorboard import notebook
# notebook.list() # View open TensorBoard instances

Known TensorBoard instances:
  - port 6015: logdir /home/ordabayev/Documents/cosmos/example/FL_3339_4444_0p8OD/runs/trackerv0.9.5/nojit/lr0.005/Adam/32/scalar (started 1:20:09 ago; pid 1320569)
  - port 6014: logdir /home/ordabayev/Documents/cosmos/example/FL_2226_3338_0p6OD_atten/runs/trackerv0.9.5-1-g3a584cf/nojit/lr0.005/Adam/32/scalar (started 1:22:17 ago; pid 1317378)
  - port 6013: logdir /home/ordabayev/Documents/cosmos/example/FL_2226_3338_0p6OD_atten/runs/trackerv0.9.5/nojit/lr0.005/Adam/32/scalar (started 1:22:37 ago; pid 1315551)
  - port 6012: logdir /home/ordabayev/Documents/cosmos/example/FL_2226_3338_0p6OD/runs/trackerv0.9.5/nojit/lr0.005/Adam/32/scalar (started 1:23:15 ago; pid 1313515)
  - port 6011: logdir /home/ordabayev/Documents/cosmos/example/GraceArticlePol2/runs/trackerv0.9.5/nojit/lr0.005/Adam/32/scalar (started 1:56:53 ago; pid 1274883)
  - port 6011: logdir /home/ordabayev/Documents/cosmos/example/FL_1_1117_0OD/runs/trackerv0.9.5/nojit/lr0.005/Adam/32/scalar 